In [ ]:
%matplotlib inline


# Create a surrogate model

In this example,
we will build a surrogate model of the Rosenbrock function
and a constraint related to an Rosenbrock-based optimization problem.


In [ ]:
from numpy import array

from gemseo import from_pickle
from gemseo import sample_disciplines
from gemseo import to_pickle
from gemseo.algos.design_space import DesignSpace
from gemseo.disciplines.analytic import AnalyticDiscipline
from gemseo.disciplines.surrogate import SurrogateDiscipline

Firstly,
we define the discipline computing the Rosenbrock function
and the Euclidean distance to the optimum:



In [ ]:
discipline = AnalyticDiscipline(
    {"z": "(1-x)**2+100*(y-x**2)**2", "c": "((x-1)**2+(y-1)**2)**0.5"},
    name="Rosenbrock",
)

Then, we create the design space:



In [ ]:
design_space = DesignSpace()
design_space.add_variable("x", lower_bound=-2.0, upper_bound=2.0, value=0.0)
design_space.add_variable("y", lower_bound=-2.0, upper_bound=2.0, value=0.0)

Then,
we sample the discipline with an optimal LHS:



In [ ]:
training_dataset = sample_disciplines([discipline], design_space, ["z", "c"], algo_name="OT_OPT_LHS", n_samples=30)

and create a test dataset for validation, using a full-factorial design of experiments:



In [ ]:
test_dataset = sample_disciplines([discipline], design_space, ["z", "c"], algo_name="OT_FULLFACT", n_samples=30**2)

before creating a surrogate discipline:



In [ ]:
surrogate_discipline = SurrogateDiscipline("RBFRegressor", training_dataset)

and using it for prediction:



In [ ]:
surrogate_discipline.execute({"x": array([1.0])})

This surrogate discipline can be used in a scenario.
The underlying regression model can also be assessed,
with the R2 measure for instance:



In [ ]:
r2 = surrogate_discipline.get_error_measure("R2Measure")

either with respect to the training dataset



In [ ]:
r2.compute_learning_measure(as_dict=True)

by cross-validation



In [ ]:
r2.compute_cross_validation_measure(as_dict=True)

or with respect to the test dataset



In [ ]:
r2.compute_test_measure(test_dataset, as_dict=True)

or with the root mean squared error:



In [ ]:
rmse = surrogate_discipline.get_error_measure("RMSEMeasure")

either with respect to the training dataset



In [ ]:
rmse.compute_learning_measure(as_dict=True)

by cross-validation



In [ ]:
rmse.compute_cross_validation_measure(as_dict=True)

or with respect to the test dataset



In [ ]:
rmse.compute_test_measure(test_dataset, as_dict=True)

Lastly,
you can save your surrogate discipline



In [ ]:
to_pickle(surrogate_discipline, "my_surrogate.pkl")

to use it later in another script:



In [ ]:
surrogate_discipline = from_pickle("my_surrogate.pkl")
surrogate_discipline.execute({"x": array([1.0])})